In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, confusion_matrix
import seaborn as sns
from google.colab import drive
import torch.nn.functional as F


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Project271B

/content/drive/.shortcut-targets-by-id/1Nr0kdWFr2lwIr68GZsixdoEXZU3DUzVg/Project271B


In [ ]:
Data = pd.read_csv("OS_data")
Data.head()

,Unnamed: 0,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,TenYearCHD
0,0,0,38,2.0,0,0.0,0.0,0,0,0,169.0,115.0,60.0,26.87,65.0,0
1,1,1,40,2.0,1,30.0,0.0,0,1,0,282.0,130.0,80.0,23.90,66.0,0
2,2,1,49,2.0,0,0.0,0.0,0,1,0,271.0,140.0,108.0,27.66,82.0,0
3,3,1,38,1.0,1,9.0,0.0,0,0,0,305.0,114.0,80.0,28.61,75.0,0
4,4,0,63,1.0,1,10.0,1.0,0,1,0,213.0,182.0,92.0,26.87,52.0,0


In [ ]:
y_train = Data['TenYearCHD']
X_train = Data.drop('TenYearCHD', axis=1)


In [ ]:
Data_test = pd.read_csv("OS_test")
Data_test.head()

,Unnamed: 0,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,TenYearCHD
0,2077,0,59,1.0,0,0.0,0.0,0,1,0,220.0,145.0,86.0,25.90,90.0,0
1,1623,1,46,1.0,1,5.0,0.0,0,0,0,221.0,125.0,88.0,24.81,72.0,0
2,3580,0,65,1.0,1,3.0,0.0,0,0,0,217.0,169.0,111.0,32.54,65.0,0
3,3837,0,42,3.0,0,0.0,0.0,0,0,0,215.0,111.0,72.0,25.38,77.0,0
4,575,1,43,1.0,1,43.0,0.0,0,0,0,215.0,122.0,76.0,26.84,85.0,0


In [ ]:
y_test = Data_test['TenYearCHD']
X_test = Data_test.drop('TenYearCHD', axis=1)


In [ ]:

#X_train = pd.read_csv("xtrain_dropped.csv")
#X_test = pd.read_csv("xest_dropped.csv")
#y_train = pd.read_csv("ytrain_dropped.csv")
#y_test = pd.read_csv("ytest_dropped.csv")

X_train = X_train.loc[:, ~X_train.columns.str.contains('^Unnamed')]
X_test = X_test.loc[:, ~X_test.columns.str.contains('^Unnamed')]
#y_train = y_train.loc[:, ~y_train.columns.str.contains('^Unnamed')]
#y_test = y_test.loc[:, ~y_test.columns.str.contains('^Unnamed')]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5756, 14) (798, 14) (5756,) (798,)


In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train.values)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test.values)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [ ]:
class NAM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NAM, self).__init__()
        
        # Define the linear combinations of networks
        self.networks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(1, hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
                # nn.Linear(hidden_size, hidden_size),
                # nn.ReLU(),
                nn.Linear(hidden_size, output_size)
            ) for i in range(input_size)
        ])
        
    def forward(self, x):
        # Calculate the output for each network and sum them up
        outputs = [self.networks[i](x[:,i].unsqueeze(1)) for i in range(x.size(1))]
        y = torch.sum(torch.stack(outputs, dim=0), dim=0)
        y = F.sigmoid(y)
        return y

In [ ]:
input_size = 15
hidden_size = 20
output_size = 1
model = NAM(input_size, hidden_size, output_size)

learning_rate = 0.01
epochs = 1000
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

criterion = nn.BCELoss()

# Generate some random input data
x = torch.randn(32, input_size)

# Calculate the output

# print(X_train.shape) 
y = model(X_train)

# y = model(x)

In [ ]:
y.shape

In [ ]:
for epoch in range(epochs+1):
  optimizer.zero_grad()
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train)
  loss.backward()
  optimizer.step()
  
  if epoch % 100 == 0:
      print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# Evaluate the train model
with torch.no_grad():
    y_pred = model(X_train)
    y_pred = torch.round(y_pred)

    accuracy = (y_pred == y_train.unsqueeze(1)).float().mean()
    auc = roc_auc_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)

    print("AUC score: {:.2f}".format(auc))
    print("F1 score: {:.2f}".format(f1))
    print(f"Accuracy: {accuracy.item():.4f}")

    plot_cm(y_train, y_pred)

In [ ]:
# Evaluate the test model
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = torch.round(y_pred)
    accuracy = (y_pred == y_test.unsqueeze(1)).float().mean()

    auc = roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("AUC score: {:.2f}".format(auc))
    print("F1 score: {:.2f}".format(f1))
    print(f"Accuracy: {accuracy.item():.4f}")

    plot_cm(y_test, y_pred)
